# Token-caching demo

Benchmark `DefaultAzureCredential.get_token()` with and without token caching on `AzureCliCredential`.

In [1]:
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()

Baseline benchmark

In [2]:
%%timeit -r 3 -n 1

access_token = credential.get_token("https://management.azure.com/.default").token

2.54 s ± 67.7 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


Add token caching

In [3]:
import json
import time
import azure.identity

def wrap_with_token_cache(object, property_name):
    token_cache = {}
    func = getattr(object, property_name)
    def wrapper(self, *args, **kwargs):
        key = json.dumps(args, sort_keys=True) + json.dumps(kwargs, sort_keys=True)
        token = token_cache.get(key, None)
        if token is None or int(time.time()) >= token.expires_on - 300:
            token = func(self, *args, **kwargs)
            token_cache[key] = token
        return token
    setattr(object, property_name, wrapper)

wrap_with_token_cache(azure.identity.AzureCliCredential, 'get_token')
wrap_with_token_cache(azure.identity.AzureCliCredential, 'get_token_info')

Token-cache benchmark

In [4]:
%%timeit -r 3 -n 1

access_token = credential.get_token("https://management.azure.com/.default").token

The slowest run took 208979.32 times longer than the fastest. This could mean that an intermediate result is being cached.
850 ms ± 1.2 s per loop (mean ± std. dev. of 3 runs, 1 loop each)
